In [2]:
import numpy as np
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import sys, rospy
import random
from os.path import expanduser
home = expanduser("~") 
sys.path.insert(3, f"{home}/catkin_ws/src/neurocontroller_database/src")
from utilities import root_database
from ROS import load_object_space_ros
from non_dominated import fast_non_dominated_sort
from neurocontroller_database.srv import load_obj_space
from distributed_individuals import select_inds, unique_inds, vect_proj, crowding_distance
from PBI import PBI
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.visualization.scatter import Scatter


In [8]:
def te():
    res = load_object_space_ros("optimized-8", return_object=True, non_dominated=True)

    print(res["id_pareto_inds"][[1,2]])
    # sel_inds = select_inds(points, get_all_data=True)
    # # print(sel_inds["num_inds"])
    
    # return unique_inds(sel_inds["distance"])
       


te()

TypeError: list indices must be integers or slices, not list

In [3]:

# def size_pop(ini, end):

#     for i in range(ini, end):
#         id_pareto = f"optimized-{i}"

#         points = load_object_space_ros_non_dom(id_pareto, non_dominated=True)

#         print(points.shape)


# size_pop(1,60)
    

In [4]:
# points = np.random.random((100,2))

# non_dom = fast_non_dominated_sort(points)
# fig = px.scatter(pd.DataFrame({"rank":non_dom, "x":points[:,0], "y":points[:,1]}), x="x", y="y", color="rank")
# fig.show()

In [5]:
ref_dirs = get_reference_directions("uniform", 3, n_partitions=3, seed=1)

# Scatter().add(ref_dirs).show()
f = px.scatter_3d(pd.DataFrame({"x":ref_dirs[:,0], "y":ref_dirs[:,1], "z":ref_dirs[:,2],"c":np.arange(ref_dirs.shape[0])}), x="x", y="y", z="z", color="c")
f.show()

In [6]:

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    if len(vector.shape) == 1:
        vector = np.array([vector])
        
    return vector / np.repeat(np.linalg.norm(vector, axis=1), vector.shape[1]).reshape(vector.shape)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    print(v1_u.shape, v2_u.shape)
    return np.arccos(np.clip(np.dot(v1_u, v2_u.T), -1.0, 1.0))

def sphere_():
    a = 15
    r = 0.2
    theta = np.radians(np.linspace(0, 360, a+1))
    phi = np.radians(np.linspace(0, 360, a+1))

    x = r * np.einsum("i,j->ij", np.cos(phi), np.sin(theta))
    y = r * np.einsum("i,j->ij", np.sin(phi), np.sin(theta))
    z = r * np.einsum("i,j->ij", np.ones(len(theta)), np.cos(theta))
    sp = np.array([x.flatten(), y.flatten(), z.flatten()])
    sp = sp.transpose(1,0) + 10


    # df_s = pd.DataFrame({"x":sp[:,0], "y":sp[:,1], "z":sp[:,2]})
    # fig = px.scatter_3d(df_s, x="x", y="y", z="z")
    # fig.show()

    return sp
    
    
def test_dist(id_pareto="", is_crowding_distance=False):
    if id_pareto == "":
        sp = sphere_()
        rank = fast_non_dominated_sort(sp)
        sp = sp[rank == 1]
    else:
        sp = load_object_space_ros_non_dom(id_pareto)

    # df_s = pd.DataFrame({"x":sp[:,0], "y":sp[:,1], "z":sp[:,2]})
    # fig = px.scatter_3d(df_s, x="x", y="y", z="z")
    # fig.show()

    inds = select_inds(sp, is_crowding_dist=is_crowding_distance, get_all_data=True)
    points = inds["points"]
    sel_inds = inds["num_inds"]
    print(sel_inds)

    # num points
    col = np.full(points.shape[0], "point")
    # inds selected
    col[sel_inds] = "sel"
    
    if not is_crowding_distance:
        ref_points = inds["ref_points"]
        points = np.vstack((points, ref_points))
        col = np.append(col, ["ref"]*ref_points.shape[0])



    df_s = pd.DataFrame({"x":points[:,0], "y":points[:,1], "z":points[:,2], "c":col})
    fig = px.scatter_3d(df_s, x="x", y="y", z="z", color="c")
    fig.show()

In [7]:

test_dist()

[11  1 10 30]


In [8]:
test_dist("optimized-8")

[ 5 10  7  6]


In [9]:
test_dist("optimized-40", is_crowding_distance=True)

[49 22  0 47]


In [10]:
# # a = np.random.random((10,2))
# a = np.array([[1,2],[3,4],[1,5],[7,1]])

# b = crowding_distance(a)
# b

In [11]:
# # prueba de la distancia perpendicular

# def test_perp_dist():
#     w = np.array([[5,2],[2,5], [6,8], [1,0],[1,1], [2,4], [6,8],[3,9], [1,5], [0,9]])
#     S = np.array([[1,2],[3,4], [7,4], [8,9]])

#     r = vect_proj(S,w)
#     a = angle_between(r[0], w)
#     print(a)

#     print(r.shape, S.shape)


#     w_dist = []
#     for i in range(r.shape[0]):
#         # projection to St
#         p_w = S - r[i]
#         # print(p_w.shape, r[i].shape, w[i].shape)
#         # perpendicular distance
#         dist_per = np.linalg.norm(p_w, axis=1)
#         w_dist.append(dist_per)
#         # print(dist_per)

#     w_dist = np.array(w_dist).T
#     print(w_dist)

#     sort_dist = np.argsort(w_dist, axis=0)
#     print(sort_dist)
#     sel_ref = np.array([0,3,5,9])

#     sel_inds = sort_dist[0,sel_ref]
#     print(sel_inds)



#     # # pos min dist
#     # pi_pos = np.argmin(w_dist, axis=1)
#     # print(pi_pos)
#     # # val of w assigned to St_i
#     # pi = w[pi_pos]
#     # print(pi)
#     # # distance of St_i assigned to w_j
#     # d = w_dist[np.arange(pi_pos.shape[0]) , pi_pos]
#     # d

#     df = pd.DataFrame({"x":S[:,0], "y":S[:,1]})
#     fig = px.scatter(df, x="x", y="y")

#     f2 = go.Scatter(x=[w[0,0]], y=[w[0,1]])
#     fig.add_trace(f2)
#     f2 = go.Scatter(x=r[0,:,0], y=r[0,:,1])
#     fig.add_trace(f2)

#     fig.add_shape(type="line", x0=0.5, y0=0, x1=0.5,y1=1, xref='x', yref='y', line=dict(color="green",))
#     fig.add_shape(type="line", x0=0, y0=0.5, x1=1,y1=0.5, xref='x', yref='y', line=dict(color="green",))
#     fig.update_yaxes(scaleratio=1,)
#     fig.update_xaxes(scaleratio=1,)
#     fig.show()

# test_dist()

In [12]:


def plot_all_obj_space(to_front, ini=1):
    number_of_colors = 61
    x = np.array([])
    y = np.array([])
    z = np.array([])
    id_p = np.array([])
    for n in range(to_front-ini+1):
        n+=ini
            
        id_pareto = f"optimized-{n}"

        points = load_object_space_ros(id_pareto)

        non_dom = fast_non_dominated_sort(points)
        points = points[non_dom == 1]

        # print(np.unique(non_dom))
        
        # color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
        #      for i in range(number_of_colors)]

        # fig = go.Figure()
        # # fig.update_layout(showlegend=True)
        # fig.add_trace(go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2],
        #                 mode='markers',
        #                 name=id_pareto,
        #                 marker=dict(color=color)),
        #                 showlegend=True)

        x=np.concatenate((x, points[:,0]))
        y=np.concatenate((y, points[:,1]))
        z=np.concatenate((z, points[:,2]))
        id_p=np.concatenate((id_p, [id_pareto]*points.shape[0]))

   
    fig = px.scatter_3d(pd.DataFrame({"id":id_p, "x":x, "y":y, "z":z}), x="x", y="y", z="z",color="id")
    fig.update_traces(marker_size = 5)
    fig.show()

plot_all_obj_space(60, ini=50)

TypeError: Invalid number of arguments, args should be ['id_pareto_front', 'solutions_number', 'normalize', 'non_dominated', 'load_test'] args are('optimized-50', 0)

In [ ]:
p1 = load_object_space_ros("optimized-20")
n1 = fast_non_dominated_sort(p1)
p1 = p1[n1 == 1]

p2 = load_object_space_ros("optimized-21")
n2 = fast_non_dominated_sort(p2)
p2 = p2[n2 == 1]

p1==p2, p1, p2


(array([[False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [ True,  True,  True],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False]]),
 array([[19.71339932,  0.74100113,  0.74744383],
        [19.95542313,  0.71517922,  0.50180127],
        [19.71459024,  0.8094635 ,  0.44623266],
        [37.36212045,  0.72026066,  0.42202899],
        [19.66965292,  0.85391661,  0.74971929],
        [22.21555944,  0.61805222,  0.68226628],
        [21.18104519,  0.63522075,  0.48859293],
        [19.64231369,  0.85810013,  0.67252839],
        [19.86933286,  0.8051749 ,  0.63972602],

In [2]:


def num_to_fileName(num_gen, is_obj_space=True):
    
    str_num = ""

    # num del archivo de generación
    if num_gen > 0 and num_gen < 10:
        str_num = f"00{num_gen}"
    if num_gen >= 10 and num_gen < 100:
        str_num = f"0{num_gen}"
    if num_gen >= 100:
        str_num = f"{num_gen}"

    if str_num == "":
        raise Exception(f"No existe el numero de generacion {num_gen}")
    else:
        if is_obj_space:
            str_num = f"obj_space_gen_{str_num}.out"
        else:
            str_num = f"var_space_gen_{str_num}.out"
            
        return str_num
    

def id_to_fileName(id_pareto, is_obj_space=True, return_id=False):
    # se separan los id (id compuesto)
    ids = id_pareto.split("_")

    path_list = []
    for id in ids:
        path_ = ""

        # si el id es de un usuario
        if id.find("/") >= 0:
            # ruta de inicio de la base de datos
            path_ = f"{root_database}/database"
            # se divide el ID
            s_id = id.split("/")
            # primer posición es el nombre del usuario
            path_ += f"/{s_id[0]}"
            # segunda posición es el ID de la población
            id_aux = s_id[1]
        else:
            path_ = f"{root_database}/database_populations"
            id_aux = id
            
        # se extrae el num gen y el nombre del folder
        s_id = id_aux.split("-")
        # pimer posición es el nombre del folder
        path_ += f"/{s_id[0]}"
        # segunda posición es el número de generación
        fileName = num_to_fileName(int(s_id[1]), is_obj_space)
        # ruta completa del archivo
        path_ += f"/{fileName}"
        
        if return_id:
            path_list.append([id, path_])
        else:
            path_list.append(path_)

    return path_list



In [5]:
id_to_fileName("optimized-1_inline12-23_afg/optimized2-1", False, False)


['/home/diego/catkin_ws/src/neurocontroller_database/database_populations/optimized/var_space_gen_001.out',
 '/home/diego/catkin_ws/src/neurocontroller_database/database_populations/inline12/var_space_gen_023.out',
 '/home/diego/catkin_ws/src/neurocontroller_database/database/afg/optimized2/var_space_gen_001.out']